In [1]:
# so we can use packages from parent directory
import sys
sys.path.append("..")

In [2]:
#import skorch
import torch
import torch.nn as nn
import nltk

import numpy as np
from monroe_data import MonroeData, MonroeDataEntry, Color # last two for reading pkl file
import caption_featurizers
from color_featurizers import ColorFeaturizer, color_phi_fourier
from experiment import FeatureHandler#, evaluate_model
import evaluation
from models_gpu import CaptionEncoder, LiteralListener, PytorchModel

%load_ext autoreload
%autoreload 2

In [59]:
import importlib
import models_gpu
importlib.reload(models_gpu)
from models_gpu import CaptionEncoder, LiteralListener, PytorchModel

In [3]:
from sklearn.model_selection import GridSearchCV, ParameterGrid

## Initialize Data, Helper Functions, and Model

In [3]:
# parameter maps:

def init_default_arg_map(feature_handler):
    global default_args_map
    default_args_map = {
        # wrappers
        'PytorchModel': {'num_epochs' : 5,
                       'optimizer' : torch.optim.Adam,
                       'lr' : 0.004,
                       'criterion': torch.nn.CrossEntropyLoss
                      },
        # modules
        'CaptionEncoder': { 'embed_dim' : 100,
                          'hidden_dim' : 100,
                          'vocab_size' : feature_handler.caption_featurizer.caption_indexer.size,
                          'color_dim' : 54
                        }
    }

def fill_default_args(model, params):
    model_default_params = {}
    # add args that should be there and filter out args that shouldn't
    for superclass in model.mro(): # mro = "method resolution order - it lists all the superclasses in order
        default_params = default_args_map.get(superclass.__name__, {})
        model_default_params = dict(default_params, **model_default_params)
        
    # filter out params with keys not in default
    params = {k: params[k] for k in params.keys() & model_default_params.keys()}
    params = dict(model_default_params, **params)
    
    return params
#     for klass in default_args_map.keys():
#         if isinstance(model, klass):
#             default_params = default_arg_map[klass]
#             params = dict(default_params, **params) # extend dict
#     return params
            

In [4]:
def retrain_model(wrapper, model, train_X, train_y, parameters, verbose=True):
    wrapper_args = fill_default_args(wrapper, parameters)
    model_args = fill_default_args(model, parameters)
    if verbose:
        print(wrapper_args, model_args)
    # create new model
    new_model = wrapper(model, **wrapper_args)
    new_model.init_model(**model_args)
    
    # train model
    new_model.fit(train_X, train_y)
    
    return new_model

In [50]:
def hyperparameter_search(search_parameters, wrapper, model, feature_handler, score_model_f):
    # ~initialization code~
    train_X, train_y, assess_X, assess_y = feature_handler # for now, for debugging
    
    best_model_score = -float("inf")
    best_model = None
    best_model_params = None
    tested_params = []

    print("Searching parameter space")
    for parameters in search_parameters:
        # retrain candidate model
        candidate_model = retrain_model(wrapper, model, train_X, train_y, parameters)
        # evaluate candidate model
        print("Evaluating Candidate Model:")
        candidate_model_score = score_model_f(candidate_model, assess_X, assess_y)
        print("Parameters: {}\tScore: {}".format(parameters, candidate_model_score))
        # store best model, score, parameters
        if candidate_model_score > best_model_score:
            best_model = candidate_model
            best_model_params = parameters
            best_model_score = candidate_model_score
            
        tested_params.append((parameters, candidate_model_score))
    
    return best_model, best_model_params, tested_params

In [5]:
def assess_model(model, assess_features, assess_targets, output_to_score):
    assess_model_outputs = model.predict(assess_features)
    assess_model_scores = [output_to_score(assess_model_outputs[i], assess_targets[i]) for i in range(len(assess_model_outputs))]
#     dev_data_synth_sm = copy.deepcopy(dev_data_synth)
   # dev_data_synth_sm.data = dev_data_synth_sm.data.head(len(assess_model_scores))
    #print(dev_data_synth_sm.data.shape)
    reg_results = evaluation.score_model(dev_data_synth, assess_model_scores, score=evaluation.Score.COMPOSITE)
    return reg_results[0] # just return the rho value

In [6]:
# test on literal listener:
train_data = MonroeData("../data/csv/train_corpus_monroe.csv", "../data/entries/train_entries_monroe.pkl")
dev_data_synth = MonroeData("../data/csv/dev_corpus_synth_10fold.csv", "../data/entries/dev_corpus_synth_10fold.pkl")

In [7]:
#nltk.download('punkt')
caption_phi = caption_featurizers.CaptionFeaturizer(tokenizer=caption_featurizers.EndingTokenizer) # Use with parameter files that end in `endings_tkn` - using endings tokenizer to separate endings like "ish" and "er"
color_phi = ColorFeaturizer(color_phi_fourier, "rgb", normalized=True)
feature_handler = FeatureHandler(train_data, dev_data_synth, caption_phi, color_phi)

output_to_score_target = lambda model_output, target: np.exp(model_output[target]) # get the model's predicted probablity at each target index and use that as the score

In [8]:
print("Initializing data")
train_X = feature_handler.train_features()
train_y = feature_handler.train_targets()
assess_X = feature_handler.test_features()
assess_y = feature_handler.test_targets()

Initializing data


In [11]:
ll = LiteralListener(CaptionEncoder)

## Grid Search

In [46]:
param_grid = {"lr":[0.0005, 0.001, 0.004, 0.01], 'hidden_dim':[50, 100, 150, 200]}

In [47]:
gs = GridSearchCV(ll, param_grid, refit=False, cv=3)

In [48]:
pg = list(ParameterGrid(param_grid))
pg

[{'hidden_dim': 50, 'lr': 0.0005},
 {'hidden_dim': 50, 'lr': 0.001},
 {'hidden_dim': 50, 'lr': 0.004},
 {'hidden_dim': 50, 'lr': 0.01},
 {'hidden_dim': 100, 'lr': 0.0005},
 {'hidden_dim': 100, 'lr': 0.001},
 {'hidden_dim': 100, 'lr': 0.004},
 {'hidden_dim': 100, 'lr': 0.01},
 {'hidden_dim': 150, 'lr': 0.0005},
 {'hidden_dim': 150, 'lr': 0.001},
 {'hidden_dim': 150, 'lr': 0.004},
 {'hidden_dim': 150, 'lr': 0.01},
 {'hidden_dim': 200, 'lr': 0.0005},
 {'hidden_dim': 200, 'lr': 0.001},
 {'hidden_dim': 200, 'lr': 0.004},
 {'hidden_dim': 200, 'lr': 0.01}]

In [9]:
init_default_arg_map(feature_handler)

In [100]:
test_params = {'color_dim':54}
fill_default_args(LiteralListener, test_params)

{'num_epochs': 5,
 'optimizer': torch.optim.adam.Adam,
 'lr': 0.004,
 'criterion': torch.nn.modules.loss.CrossEntropyLoss}

In [36]:
fill_default_args(CaptionEncoder, test_params)

{'embed_dim': 100, 'hidden_dim': 100, 'vocab_size': 974, 'color_dim': 54}

In [39]:
from functools import partial
literal_listener_assess_model = partial(assess_model, output_to_score=output_to_score_target)

In [105]:
import importlib
import models_gpu
importlib.reload(models_gpu)
from models_gpu import CaptionEncoder, LiteralListener, PytorchModel

In [106]:
x = hyperparameter_search(pg, LiteralListener, CaptionEncoder, [train_X, train_y, assess_X, assess_y], 
                          literal_listener_assess_model)

Searching parameter space
{'num_epochs': 5, 'optimizer': <class 'torch.optim.adam.Adam'>, 'lr': 0.0005, 'criterion': <class 'torch.nn.modules.loss.CrossEntropyLoss'>} {'embed_dim': 100, 'hidden_dim': 50, 'vocab_size': 974, 'color_dim': 54}
---EPOCH 0---
0m 0s (0:0 0.00%) 0.0002
0m 7s (0:1000 7.90%) 1.0664
0m 16s (0:2000 15.79%) 0.8434
0m 25s (0:3000 23.69%) 0.8224
0m 36s (0:4000 31.58%) 0.7913
0m 47s (0:5000 39.48%) 0.8518
0m 58s (0:6000 47.37%) 0.6619
1m 9s (0:7000 55.27%) 0.5640
1m 20s (0:8000 63.17%) 0.6580
1m 32s (0:9000 71.06%) 0.5458
1m 44s (0:10000 78.96%) 0.5053
1m 55s (0:11000 86.85%) 0.4923
2m 7s (0:12000 94.75%) 0.5438
AFTER EPOCH 2999 - AVERAGE VALIDATION LOSS: 0.6549942147831623
---EPOCH 1---
2m 19s (1:0 0.00%) 0.0001
2m 30s (1:1000 7.90%) 0.4751
2m 42s (1:2000 15.79%) 0.5880
2m 54s (1:3000 23.69%) 0.5820
3m 7s (1:4000 31.58%) 0.6049
3m 18s (1:5000 39.48%) 0.6560
3m 31s (1:6000 47.37%) 0.4809
3m 43s (1:7000 55.27%) 0.4008
3m 55s (1:8000 63.17%) 0.5104
4m 7s (1:9000 71.06%)

11m 48s (4:2000 15.79%) 2.3594
12m 3s (4:3000 23.69%) 2.5100
12m 17s (4:4000 31.58%) 2.2293
12m 31s (4:5000 39.48%) 2.5515
12m 46s (4:6000 47.37%) 2.1915
13m 0s (4:7000 55.27%) 1.9311
13m 15s (4:8000 63.17%) 2.1952
13m 30s (4:9000 71.06%) 2.2692
13m 46s (4:10000 78.96%) 2.2464
14m 0s (4:11000 86.85%) 2.2088
14m 15s (4:12000 94.75%) 2.1190
AFTER EPOCH 2999 - AVERAGE VALIDATION LOSS: 2.2986788938223386
Evaluating Candidate Model:
Got here to composite score
Parameters: {'hidden_dim': 50, 'lr': 0.004}	Score: 0.7490148055266912
{'num_epochs': 5, 'optimizer': <class 'torch.optim.adam.Adam'>, 'lr': 0.01, 'criterion': <class 'torch.nn.modules.loss.CrossEntropyLoss'>} {'embed_dim': 100, 'hidden_dim': 50, 'vocab_size': 974, 'color_dim': 54}
---EPOCH 0---
0m 0s (0:0 0.00%) 0.0004
0m 9s (0:1000 7.90%) 15.1187
0m 20s (0:2000 15.79%) 29.7664
0m 33s (0:3000 23.69%) 36.4285
0m 47s (0:4000 31.58%) 28.2213
1m 2s (0:5000 39.48%) 25.2305
1m 18s (0:6000 47.37%) 21.8959
1m 35s (0:7000 55.27%) 18.2570
1m 52

10m 21s (3:1000 7.90%) 0.4896
10m 38s (3:2000 15.79%) 0.6794
10m 56s (3:3000 23.69%) 0.5240
11m 13s (3:4000 31.58%) 0.7174
11m 30s (3:5000 39.48%) 0.6943
11m 46s (3:6000 47.37%) 0.4759
12m 3s (3:7000 55.27%) 0.4517
12m 20s (3:8000 63.17%) 0.5342
12m 37s (3:9000 71.06%) 0.5012
12m 55s (3:10000 78.96%) 0.4527
13m 12s (3:11000 86.85%) 0.3822
13m 30s (3:12000 94.75%) 0.4906
AFTER EPOCH 2999 - AVERAGE VALIDATION LOSS: 0.8544271445252815
---EPOCH 4---
13m 46s (4:0 0.00%) 0.0000
14m 3s (4:1000 7.90%) 0.5402
14m 22s (4:2000 15.79%) 0.4960
14m 39s (4:3000 23.69%) 0.5377
14m 56s (4:4000 31.58%) 0.7105
15m 13s (4:5000 39.48%) 0.7111
15m 31s (4:6000 47.37%) 0.4058
15m 48s (4:7000 55.27%) 0.3788
16m 5s (4:8000 63.17%) 0.4714
16m 22s (4:9000 71.06%) 0.4722
16m 41s (4:10000 78.96%) 0.4700
16m 59s (4:11000 86.85%) 0.4613
17m 16s (4:12000 94.75%) 0.4238
AFTER EPOCH 2999 - AVERAGE VALIDATION LOSS: 0.9930198964091542
Evaluating Candidate Model:
Got here to composite score
Parameters: {'hidden_dim': 100, 

7m 37s (1:11000 86.85%) 0.3868
7m 58s (1:12000 94.75%) 0.4836
AFTER EPOCH 2999 - AVERAGE VALIDATION LOSS: 0.6148726755145877
---EPOCH 2---
8m 17s (2:0 0.00%) 0.0002
8m 37s (2:1000 7.90%) 0.3945
8m 58s (2:2000 15.79%) 0.4509
9m 19s (2:3000 23.69%) 0.4958
9m 39s (2:4000 31.58%) 0.5667
10m 0s (2:5000 39.48%) 0.4991
10m 22s (2:6000 47.37%) 0.3976
10m 43s (2:7000 55.27%) 0.3481
11m 4s (2:8000 63.17%) 0.4293
11m 25s (2:9000 71.06%) 0.3487
11m 47s (2:10000 78.96%) 0.3095
12m 9s (2:11000 86.85%) 0.3679
12m 31s (2:12000 94.75%) 0.4388
AFTER EPOCH 2999 - AVERAGE VALIDATION LOSS: 0.6299160175719686
---EPOCH 3---
12m 50s (3:0 0.00%) 0.0001
13m 11s (3:1000 7.90%) 0.3600
13m 34s (3:2000 15.79%) 0.4155
13m 55s (3:3000 23.69%) 0.4922
14m 17s (3:4000 31.58%) 0.5317
14m 39s (3:5000 39.48%) 0.4140
15m 1s (3:6000 47.37%) 0.3332
15m 22s (3:7000 55.27%) 0.3139
15m 45s (3:8000 63.17%) 0.3883
16m 7s (3:9000 71.06%) 0.3376
16m 29s (3:10000 78.96%) 0.2880
16m 49s (3:11000 86.85%) 0.3465
17m 11s (3:12000 94.75%)

3m 3s (0:7000 55.27%) 208.4946
3m 30s (0:8000 63.17%) 238.4550
3m 59s (0:9000 71.06%) 252.7559
4m 29s (0:10000 78.96%) 281.6474
4m 57s (0:11000 86.85%) 345.0292
5m 28s (0:12000 94.75%) 254.9305
AFTER EPOCH 2999 - AVERAGE VALIDATION LOSS: 287.51719832531313
---EPOCH 1---
6m 0s (1:0 0.00%) 1.0586
6m 33s (1:1000 7.90%) 208.8056
7m 10s (1:2000 15.79%) 242.7647
7m 47s (1:3000 23.69%) 300.9697
8m 27s (1:4000 31.58%) 295.7218
9m 10s (1:5000 39.48%) 328.5393
9m 51s (1:6000 47.37%) 247.4035
10m 32s (1:7000 55.27%) 279.6439
11m 16s (1:8000 63.17%) 377.0985
12m 2s (1:9000 71.06%) 297.9311
12m 50s (1:10000 78.96%) 336.7798
13m 35s (1:11000 86.85%) 251.6695
14m 24s (1:12000 94.75%) 312.2036
AFTER EPOCH 2999 - AVERAGE VALIDATION LOSS: 302.9091311665326
---EPOCH 2---
15m 24s (2:0 0.00%) 0.8338
16m 14s (2:1000 7.90%) 300.5228
17m 10s (2:2000 15.79%) 412.1149
18m 9s (2:3000 23.69%) 470.9999
19m 15s (2:4000 31.58%) 421.3149
20m 23s (2:5000 39.48%) 417.7132
21m 27s (2:6000 47.37%) 372.9073
22m 30s (2:700

Got here to composite score
Parameters: {'hidden_dim': 200, 'lr': 0.001}	Score: 0.9496320532069575
{'num_epochs': 5, 'optimizer': <class 'torch.optim.adam.Adam'>, 'lr': 0.004, 'criterion': <class 'torch.nn.modules.loss.CrossEntropyLoss'>} {'embed_dim': 100, 'hidden_dim': 200, 'vocab_size': 974, 'color_dim': 54}
---EPOCH 0---
0m 0s (0:0 0.00%) 0.0011
0m 22s (0:1000 7.90%) 6.2914
0m 48s (0:2000 15.79%) 4.4022
1m 14s (0:3000 23.69%) 3.0461
1m 43s (0:4000 31.58%) 2.4873
2m 11s (0:5000 39.48%) 2.7194
2m 41s (0:6000 47.37%) 2.5321
3m 13s (0:7000 55.27%) 3.5300
3m 44s (0:8000 63.17%) 3.9138
4m 17s (0:9000 71.06%) 3.7037
4m 54s (0:10000 78.96%) 2.1528
5m 28s (0:11000 86.85%) 2.5934
6m 5s (0:12000 94.75%) 3.0101
AFTER EPOCH 2999 - AVERAGE VALIDATION LOSS: 6.274906044489316
---EPOCH 1---
6m 39s (1:0 0.00%) 0.0119
7m 20s (1:1000 7.90%) 4.3519
8m 2s (1:2000 15.79%) 7.0834
8m 42s (1:3000 23.69%) 4.7707
9m 25s (1:4000 31.58%) 5.1719
10m 6s (1:5000 39.48%) 3.3804
10m 52s (1:6000 47.37%) 2.2259
11m 37

In [107]:
best_model, best_model_params, tested_params = x
best_model.save_model('../model/best_literal_listener.params')
print(best_model_params, tested_params)

{'hidden_dim': 100, 'lr': 0.0005} [({'hidden_dim': 50, 'lr': 0.0005}, 0.9580292528683316), ({'hidden_dim': 50, 'lr': 0.001}, 0.9590177684352623), ({'hidden_dim': 50, 'lr': 0.004}, 0.7490148055266912), ({'hidden_dim': 50, 'lr': 0.01}, 0.8119920657704628), ({'hidden_dim': 100, 'lr': 0.0005}, 0.9624430398406268), ({'hidden_dim': 100, 'lr': 0.001}, 0.9579455553391637), ({'hidden_dim': 100, 'lr': 0.004}, 0.4380851056933659), ({'hidden_dim': 100, 'lr': 0.01}, 0.8977222373675259), ({'hidden_dim': 150, 'lr': 0.0005}, 0.9603001451994843), ({'hidden_dim': 150, 'lr': 0.001}, 0.9542372978868268), ({'hidden_dim': 150, 'lr': 0.004}, 0.8062140998933828), ({'hidden_dim': 150, 'lr': 0.01}, 0.9200245680675747), ({'hidden_dim': 200, 'lr': 0.0005}, 0.9540587815829551), ({'hidden_dim': 200, 'lr': 0.001}, 0.9496320532069575), ({'hidden_dim': 200, 'lr': 0.004}, 0.44983413200281136), ({'hidden_dim': 200, 'lr': 0.01}, 0.900607774498584)]


## Load and Evaluate Model Multiple Times

In [10]:
l1 = LiteralListener(CaptionEncoder)
l1.init_model(embed_dim=100, hidden_dim=100, vocab_size=feature_handler.caption_featurizer.caption_indexer.size,
             color_dim=54)
l1.load_model('../model/best_literal_listener.params')

In [11]:
best_params = {'lr':0.0005, 'hidden_dim':100}
fill_default_args(LiteralListener, best_params)
fill_default_args(CaptionEncoder, best_params)

{'embed_dim': 100, 'hidden_dim': 100, 'vocab_size': 974, 'color_dim': 54}

In [12]:
def sample_trained_model(parameters, wrapper, model, train_dev_data, score_model_f, num_samples=10):
    # ~initialization code~
    train_X, train_y, assess_X, assess_y = train_dev_data # for now, for debugging
    
    all_model_scores = []

    for i in range(num_samples):
        # retrain candidate model
        candidate_model = retrain_model(wrapper, model, train_X, train_y, parameters, verbose=False)
        # evaluate candidate model
        print("Evaluating Candidate Model:")
        candidate_model_score = score_model_f(candidate_model, assess_X, assess_y)
        print("\t\tCorrelation\tMean\tStd")
        print("Iteration #{} Score: {}".format(i, candidate_model_score))
        
        candidate_model.save_model("../sampling/sample_{}.params".format(i))
        all_model_scores.append(candidate_model_score)
    
    return all_model_scores

In [13]:
def assess_model_mean_acc(model, assess_features, assess_targets, output_to_score):
    assess_model_outputs = model.predict(assess_features)
    assess_model_scores = [output_to_score(assess_model_outputs[i], assess_targets[i]) for i in range(len(assess_model_outputs))]
    accuracy=[np.argmax(assess_model_outputs[i]) == assess_targets[i] for i in range(len(assess_model_outputs))]
    
    reg_results = evaluation.score_model(dev_data_synth, assess_model_scores, score=evaluation.Score.COMPOSITE)
    return reg_results[0], np.mean(accuracy)

In [11]:
# try running once
assess_model_mean_acc(l1, assess_X, assess_y, output_to_score_target)

Got here to composite score


(0.9624430393932746, 0.46259187620889747)

In [14]:
from functools import partial
literal_listener_assess_model_mean_acc = partial(assess_model_mean_acc, output_to_score=output_to_score_target)

In [15]:
sample_trained_model(best_params, LiteralListener, CaptionEncoder, [train_X, train_y, assess_X, assess_y], 
                     literal_listener_assess_model_mean_acc, num_samples=10)

---EPOCH 0---
0m 0s (0:0 0.00%) 0.0010
0m 11s (0:1000 7.90%) 1.2594
0m 23s (0:2000 15.79%) 0.8863
0m 35s (0:3000 23.69%) 0.8454
0m 49s (0:4000 31.58%) 0.7943
1m 2s (0:5000 39.48%) 0.8332
1m 16s (0:6000 47.37%) 0.6568
1m 31s (0:7000 55.27%) 0.5985
1m 46s (0:8000 63.17%) 0.6506
2m 0s (0:9000 71.06%) 0.5871
2m 15s (0:10000 78.96%) 0.5298
2m 29s (0:11000 86.85%) 0.4956
2m 43s (0:12000 94.75%) 0.5440
AFTER EPOCH 2999 - AVERAGE VALIDATION LOSS: 0.7282057183378535
---EPOCH 1---
2m 58s (1:0 0.00%) 0.0002
3m 14s (1:1000 7.90%) 0.4876
3m 33s (1:2000 15.79%) 0.5946
3m 48s (1:3000 23.69%) 0.6027
4m 3s (1:4000 31.58%) 0.6324
4m 20s (1:5000 39.48%) 0.6053
4m 35s (1:6000 47.37%) 0.4971
4m 51s (1:7000 55.27%) 0.4089
5m 6s (1:8000 63.17%) 0.5007
5m 21s (1:9000 71.06%) 0.4574
5m 36s (1:10000 78.96%) 0.3665
5m 51s (1:11000 86.85%) 0.3831
6m 6s (1:12000 94.75%) 0.4772
AFTER EPOCH 2999 - AVERAGE VALIDATION LOSS: 0.6434896682112626
---EPOCH 2---
6m 25s (2:0 0.00%) 0.0002
6m 52s (2:1000 7.90%) 0.3777
7m 21s 

0m 10s (0:1000 7.90%) 1.3059
0m 22s (0:2000 15.79%) 0.8579
0m 34s (0:3000 23.69%) 0.8195
0m 48s (0:4000 31.58%) 0.7825
1m 1s (0:5000 39.48%) 0.8340
1m 15s (0:6000 47.37%) 0.6322
1m 29s (0:7000 55.27%) 0.5749
1m 44s (0:8000 63.17%) 0.6759
1m 58s (0:9000 71.06%) 0.5505
2m 13s (0:10000 78.96%) 0.5214
2m 27s (0:11000 86.85%) 0.4986
2m 42s (0:12000 94.75%) 0.5474
AFTER EPOCH 2999 - AVERAGE VALIDATION LOSS: 0.6604023328418833
---EPOCH 1---
2m 56s (1:0 0.00%) 0.0013
3m 12s (1:1000 7.90%) 0.4706
3m 29s (1:2000 15.79%) 0.5674
3m 44s (1:3000 23.69%) 0.6003
3m 59s (1:4000 31.58%) 0.6259
4m 14s (1:5000 39.48%) 0.6260
4m 29s (1:6000 47.37%) 0.4468
4m 44s (1:7000 55.27%) 0.3982
4m 59s (1:8000 63.17%) 0.5025
5m 14s (1:9000 71.06%) 0.4442
5m 30s (1:10000 78.96%) 0.4069
5m 44s (1:11000 86.85%) 0.3993
5m 59s (1:12000 94.75%) 0.4337
AFTER EPOCH 2999 - AVERAGE VALIDATION LOSS: 0.5846138624934785
---EPOCH 2---
6m 14s (2:0 0.00%) 0.0002
6m 29s (2:1000 7.90%) 0.3660
6m 44s (2:2000 15.79%) 0.4709
7m 0s (2:300

0m 17s (0:1000 7.90%) 1.2487
0m 39s (0:2000 15.79%) 0.8789
1m 1s (0:3000 23.69%) 0.8324
1m 25s (0:4000 31.58%) 0.7843
1m 49s (0:5000 39.48%) 0.8487
2m 15s (0:6000 47.37%) 0.6348
2m 40s (0:7000 55.27%) 0.5550
3m 7s (0:8000 63.17%) 0.6883
3m 33s (0:9000 71.06%) 0.5421
4m 0s (0:10000 78.96%) 0.5535
4m 26s (0:11000 86.85%) 0.5185
4m 53s (0:12000 94.75%) 0.5384
AFTER EPOCH 2999 - AVERAGE VALIDATION LOSS: 0.6825501924546119
---EPOCH 1---
5m 20s (1:0 0.00%) 0.0001
5m 47s (1:1000 7.90%) 0.4668
6m 16s (1:2000 15.79%) 0.5586
6m 43s (1:3000 23.69%) 0.5960
7m 12s (1:4000 31.58%) 0.6127
7m 39s (1:5000 39.48%) 0.6269
8m 6s (1:6000 47.37%) 0.4585
8m 34s (1:7000 55.27%) 0.3608
9m 2s (1:8000 63.17%) 0.5182
9m 30s (1:9000 71.06%) 0.4449
9m 59s (1:10000 78.96%) 0.4003
10m 27s (1:11000 86.85%) 0.4152
10m 54s (1:12000 94.75%) 0.4149
AFTER EPOCH 2999 - AVERAGE VALIDATION LOSS: 0.5952420088993893
---EPOCH 2---
11m 22s (2:0 0.00%) 0.0006
11m 48s (2:1000 7.90%) 0.3767
12m 17s (2:2000 15.79%) 0.4670
12m 45s (2:

0m 18s (0:1000 7.90%) 1.2577
0m 40s (0:2000 15.79%) 0.8507
1m 2s (0:3000 23.69%) 0.8198
1m 26s (0:4000 31.58%) 0.7902
1m 50s (0:5000 39.48%) 0.8329
2m 16s (0:6000 47.37%) 0.6154
2m 42s (0:7000 55.27%) 0.5793
3m 7s (0:8000 63.17%) 0.6573
3m 33s (0:9000 71.06%) 0.5229
4m 0s (0:10000 78.96%) 0.5318
4m 28s (0:11000 86.85%) 0.5184
4m 54s (0:12000 94.75%) 0.5507
AFTER EPOCH 2999 - AVERAGE VALIDATION LOSS: 0.6372529485517121
---EPOCH 1---
5m 22s (1:0 0.00%) 0.0005
5m 48s (1:1000 7.90%) 0.4542
6m 17s (1:2000 15.79%) 0.5749
6m 44s (1:3000 23.69%) 0.6419
7m 11s (1:4000 31.58%) 0.6223
7m 38s (1:5000 39.48%) 0.6040
8m 6s (1:6000 47.37%) 0.4437
8m 32s (1:7000 55.27%) 0.4193
9m 0s (1:8000 63.17%) 0.5007
9m 27s (1:9000 71.06%) 0.4002
9m 55s (1:10000 78.96%) 0.4149
10m 22s (1:11000 86.85%) 0.4593
10m 49s (1:12000 94.75%) 0.4456
AFTER EPOCH 2999 - AVERAGE VALIDATION LOSS: 0.5983944939464575
---EPOCH 2---
11m 17s (2:0 0.00%) 0.0001
11m 49s (2:1000 7.90%) 0.3587
12m 18s (2:2000 15.79%) 0.4743
12m 45s (2:

[(0.9540583513077912, 0.46),
 (0.9590163878691865, 0.46174081237911024),
 (0.9587481489768644, 0.4630174081237911),
 (0.9566161098824271, 0.4609284332688588),
 (0.9562418201877905, 0.4602707930367505),
 (0.9545501622959333, 0.45547388781431336),
 (0.958216009901539, 0.46135396518375243),
 (0.960408133748805, 0.4630560928433269),
 (0.9588654435565022, 0.4583752417794971),
 (0.9592406897047975, 0.46263056092843324)]